In [3]:
import pandas
import os

In [4]:
import utils

In [5]:
import os
from math import radians, cos, sin, asin, sqrt
import sys
import time

import sys
sys.path.append('/Users/gustaveronteix/Documents/Projets/Projets Code/batch7_beges/datasets/chorus-dt')

import requests
import dill
import urllib.parse
import pandas as pd
from tqdm import tqdm
from utils.config import CONFIG

from utils import dataset
from utils.resolvers import HardcodesResolver, GeocodingApiResolver

tqdm.pandas()


def compute_distance(df, n):

    coord_lon_str = 'coords_place_'+str(n)+'_lon'
    coord_lat_str = 'coords_place_'+str(n)+'_lat'
    coord_lon_p_str = 'coords_place_'+str(n-1)+'_lon'
    coord_lat_p_str = 'coords_place_'+str(n-1)+'_lat'

    lon0 = df[coord_lon_str]
    lat0 = df[coord_lat_str]
    lon1 = df[coord_lon_p_str]
    lat1 = df[coord_lat_p_str]

    lon0 = np.deg2rad(lon0)
    lon1 = np.deg2rad(lon1)
    lat0 = np.deg2rad(lat0)
    lat1 = np.deg2rad(lat1)

    # haversine formula
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = np.sin((dlat)/2) ** 2 + np.cos(lat1) * np.cos(lat0) * np.sin((dlon)/ 2)** 2

    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in miles. Use 6371 for kilometers

    return c*r


if __name__ == "__main__":
    data = dataset.load_data("./data/raw/sample-avion-train.csv")
    places, trips = dataset.get_places_and_trips(data)

    # Splitting codes / name
    places[["code_1", "name", "temp", "code_2"]] = places["place"].str.extract(
        "([^-]*)\s-\s([^\[]*)\s?([\[](.*)[\]])?", expand=True
    )
    places.drop(columns=["temp"], inplace=True)

    places["resolved"] = False
    places["resolved_through_uic_code"] = False
    places["resolved_through_insee_code"] = False
    places["resolved_through_tvs_code"] = False
    places["resolved_through_iata_code"] = False
    places["resolved_through_gmap_api"] = False
    places["lon"] = None
    places["address"] = ""
    places["lat"] = None

    hardcode_resolver = HardcodesResolver()
    places = places.progress_apply(hardcode_resolver.resolve, axis=1)

    # Initiate geocoder
    GMAP_API_KEY = CONFIG["api_keys"]["gmap"]
    geocoding_api_resolver = GeocodingApiResolver(api_name="gmap", api_key=GMAP_API_KEY)

    # Apply the geocoder with delay using the rate limiter:
    places.loc[~places["resolved"], :] = places[~places["resolved"]].progress_apply(
        geocoding_api_resolver.resolve, axis=1
    )
    geocoding_api_resolver.save_cache()
    for c in places:
        if "resolved_through_" in c:
            print(c, places[c].sum())

    places["lat"] = places["lat"].astype(float)
    places["lon"] = places["lon"].astype(float)
    places_dict = places.set_index("place").to_dict()

    # when analyzing plane trips, doesn't work if
    # 'no stop' in trips columns. (ie. always since we programed
    # plane trips this way.)

    for place_index in [0, 1, 2]:
        trips["coords_place_%d_lat" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: None if x == 'No stop' else places_dict["lat"][x])
        trips["coords_place_%d_lon" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: None if x == 'No stop' else places_dict["lon"][x])
        trips["coords_place_%d" % place_index] = (
            trips["coords_place_%d_lon" % place_index].astype(str)
            + ";"
            + trips["coords_place_%d_lat" % place_index].astype(str)
        )
        trips["place_%d_count" % place_index] = trips[
            "trip_place_%d" % place_index
        ].apply(lambda x: None if x == 'No stop' else places_dict["total"][x])

    trips["distance_1"] = compute_distance(trips, 1).fillna(0)
    trips["distance_2"] = compute_distance(trips, 2).fillna(0)
    trips["distance"] = trips["distance_1"] + trips["distance_2"]

    places.to_csv("./data/clean/places.csv", index=False)
    trips.to_csv("./data/clean/trips.csv", index=False)


/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


./data/raw/sample-avion-train.csv


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-14331be94858>", line 52, in <module>
    places, trips = dataset.get_places_and_trips(data)
  File "/Users/gustaveronteix/Documents/Projets/Projets Code/batch7_beges/datasets/chorus-dt/utils/dataset.py", line 59, in get_places_and_trips
    current_data["prestation_type"].str.contains("|".join(prestation_type_filters))
  File "/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2791, in __getitem__
    return self._getitem_bool_array(key)
  File "/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 2845, in _getitem_bool_array
    return self._take_with_is_copy(indexer, axis=0)
  File "/Users/gustaveronteix/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.

KeyboardInterrupt: 

In [8]:
df = pandas.read_csv("./data/raw/sample-avion-train.csv", delimiter="	")

In [16]:
df[df['Type de prestation'].str.contains('TC - ')]

,N° de l'OM,"Mode de réservation (online, offline)",Structure,Date de début mission,Date de fin mission,Statut,Lieu de départ,Lieu d'arrivée,Type de prestation,Lieu étape,Nombre de prestations,Nombre d'OM,Nombre de jours,Coût des prestations
17186,LRDXC,offline,DG010-07000 - AC/SG/SPSSI,14/1/2019,18/2/2019,T - Traité,78551 - SAINT-GERMAIN-EN-LAYE,DEP075 - PARIS,TC - Transport en commun,DEP075 - PARIS,1,1,36,NaN
24219,LVZBT,offline,DG090-01000 - CGEDD/BEATT,20/3/2019,20/3/2019,V - Validé,75056 - PARIS,75056 - PARIS,TC - Transport en commun,75056 - PARIS,1,1,1,NaN
31806,MAFVC,offline,DG060-04000 - DGPR/SRNH,12/6/2019,12/6/2019,T - Traité,92062 - PUTEAUX,75056 - PARIS,TC - Transport en commun,75056 - PARIS,1,1,1,2.98
32390,MAPQ3,offline,DG090-02000 - CGEDD/CGEDD,14/6/2019,14/6/2019,F - Annulé,92062 - PUTEAUX,77288 - MELUN,TC - Transport en commun,77288 - MELUN,1,1,1,NaN
35215,MB4WS,offline,DG010-05000 - AC/SG/DAJ,21/6/2019,21/6/2019,V - Validé,92062 - PUTEAUX,95127 - CERGY,TC - Transport en commun,95127 - CERGY,1,1,1,NaN
35220,MB4ZQ,offline,DG010-05000 - AC/SG/DAJ,21/6/2019,21/6/2019,V - Validé,92050 - NANTERRE,95127 - CERGY,TC - Transport en commun,95127 - CERGY,1,1,1,NaN
35471,MCBZY,offline,DG090-02000 - CGEDD/CGEDD,20/6/2019,20/6/2019,T - Traité,92049 - MONTROUGE,IA_LBG - LE BOURGET,TC - Transport en commun,IA_LBG - LE BOURGET,1,1,1,10.5
38158,MD9C3,offline,DG060-04000 - DGPR/SRNH,24/6/2019,24/6/2019,T - Traité,92062 - PUTEAUX,75056 - PARIS,TC - Transport en commun,75056 - PARIS,1,1,1,3.73
41408,MHAA7,offline,DG090-02000 - CGEDD/CGEDD,17/7/2019,17/7/2019,T - Traité,92064 - SAINT-CLOUD,92060 - PLESSIS-ROBINSON,TC - Transport en commun,92060 - PLESSIS-ROBINSON,1,1,1,9.43
49579,MMRDP,offline,DG010-01200 - AC/SG/DRH/DRH SERVICES,10/9/2019,10/9/2019,G - Attente de validation Gestionnaire,92050 - NANTERRE,75056 - PARIS,TC - Transport en commun,75056 - PARIS,1,1,1,NaN


In [ ]:
d